In [2]:
import nussl
import torch
from nussl.datasets import transforms as nussl_tfm
from models.Waveform import Waveform
#from utils.models import MaskInference
from utils import utils, data
from pathlib import Path

In [3]:
#data.prepare_musdb('~/audio_isolation/data/tutorial/')

In [4]:
utils.logger()
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
MAX_MIXTURES = int(1e8) # We'll set this to some impossibly high number for on the fly mixing.

stft_params = nussl.STFTParams(window_length=512, hop_length=128)

tfm = nussl_tfm.Compose([
    nussl_tfm.SumSources([['bass', 'drums', 'other']]),
    nussl_tfm.GetAudio(),
    #nussl_tfm.MagnitudeSpectrumApproximation(),
    nussl_tfm.IndexSources('source_audio', 1),
    nussl_tfm.ToSeparationModel(),
])

train_folder = "~/audio_isolation/data/tutorial/train"
val_folder = "~/audio_isolation/data/tutorial/test"

train_data = data.on_the_fly(stft_params, n_channels=2, transform=tfm, 
    fg_path=train_folder, num_mixtures=MAX_MIXTURES, coherent_prob=1.0)
train_dataloader = torch.utils.data.DataLoader(
    train_data, num_workers=1, batch_size=10)

val_data = data.on_the_fly(stft_params, n_channels=2, transform=tfm, 
    fg_path=val_folder, num_mixtures=10, coherent_prob=1.0)
val_dataloader = torch.utils.data.DataLoader(
    val_data, num_workers=1, batch_size=10)

In [5]:
model = Waveform.build(1025, 2, 50, 2, True, 0.3, 1, 2048, 512)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nussl.ml.train.loss.L1Loss()

In [6]:
model.config

{'name': 'Waveform',
 'modules': {'model': {'class': 'Waveform',
   'args': {'num_features': 1025,
    'num_audio_channels': 2,
    'hidden_size': 50,
    'num_layers': 2,
    'bidirectional': True,
    'dropout': 0.3,
    'embedding_size': 1,
    'num_filters': 2048,
    'hop_length': 512,
    'window_type': 'sqrt_hann',
    'activation': ['sigmoid', 'unit_norm']},
   'module_snapshot': "class Waveform(nn.Module):\n    def __init__(self, num_features, num_audio_channels, hidden_size,\n                num_layers, bidirectional, dropout, embedding_size, \n                num_filters, hop_length, window_type='sqrt_hann', # New STFT parameters\n                activation=['sigmoid', 'unit_norm']):\n        super().__init__()\n        \n        self.stft = STFT(\n            num_filters, \n            hop_length=hop_length, \n            window_type=window_type\n        )\n        \n        self.amplitude_to_db = AmplitudeToDB()\n        self.input_normalization = BatchNorm(num_features)\n

In [7]:
# nf = stft_params.window_length // 2 + 1
# model = Waveform.build(nf, 1, 50, 1, True, 0.0, 1, 'sigmoid')
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
# loss_fn = nussl.ml.train.loss.L1Loss()

def train_step(engine, batch):
    optimizer.zero_grad()
    output = model(batch) # forward pass
    loss = loss_fn(
        output['estimates'],
        batch['source_audio']
    )
    
    loss.backward() # backwards + gradient step
    optimizer.step()
    
    loss_vals = {
        'L1Loss': loss.item(),
        'loss': loss.item()
    }
    
    return loss_vals

def val_step(engine, batch):
    with torch.no_grad():
        output = model(batch) # forward pass
    loss = loss_fn(
        output['estimates'],
        batch['source_audio']
    )    
    loss_vals = {
        'L1Loss': loss.item(), 
        'loss': loss.item()
    }
    return loss_vals

# Create the engines
trainer, validator = nussl.ml.train.create_train_and_validation_engines(
    train_step, val_step, device=DEVICE
)

# We'll save the output relative to this notebook.
output_folder = Path('.').absolute()

# Adding handlers from nussl that print out details about model training
# run the validation step, and save the models.
nussl.ml.train.add_stdout_handler(trainer, validator)
nussl.ml.train.add_validate_and_checkpoint(output_folder, model, 
    optimizer, train_data, trainer, val_dataloader, validator)

trainer.run(
    train_dataloader, 
    epoch_length=10, 
    max_epochs=25
)

04/18/2023 05:59:45 PM | engine.py:874 Engine run starting with max_epochs=25.
04/18/2023 06:00:17 PM | engine.py:874 Engine run starting with max_epochs=1.
04/18/2023 06:00:22 PM | engine.py:972 Epoch[1] Complete. Time taken: 00:00:05.433
04/18/2023 06:00:22 PM | engine.py:988 Engine run complete. Time taken: 00:00:05.443
04/18/2023 06:00:28 PM | trainer.py:311 

EPOCH SUMMARY 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
- Epoch number: 0001 / 0025 
- Training loss:   0.087930 
- Validation loss: 0.101172 
- Epoch took: 0:00:42.475676 
- Time since start: 0:00:42.475693 
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
Saving to /Users/dev/audio_isolation/checkpoints/best.model.pth. 
Output @ /Users/dev/audio_isolation 

04/18/2023 06:00:28 PM | engine.py:972 Epoch[1] Complete. Time taken: 00:00:37.345
04/18/2023 06:00:51 PM | engine.py:874 Engine run starting with max_epochs=1.
04/18/2023 06:00:57 PM | engine.py:972 Epoch[1] Complete. Time taken: 00:00:05.529
04/18/2023 06:00:57 PM | engine.py:988 Engine run com

KeyboardInterrupt: 

In [ ]:
separator = nussl.separation.deep.DeepMaskEstimation(
    nussl.AudioSignal(), model_path='checkpoints/best.model.pth',
    device=DEVICE,
)

In [ ]:
separator = nussl.separation.deep.DeepAudioEstimation(
    nussl.AudioSignal(), model_path='checkpoints/best.model.pth',
    device=DEVICE,
)

/Users/dev/miniforge3/envs/audio-isolation/lib/python3.10/site-packages/nussl/separation/base/separation_base.py:73: UserWarning: input_audio_signal has no data!
  warnings.warn('input_audio_signal has no data!')
/Users/dev/miniforge3/envs/audio-isolation/lib/python3.10/site-packages/nussl/core/audio_signal.py:455: UserWarning: Initializing STFT with data that is non-complex. This might lead to weird results!
  warnings.warn('Initializing STFT with data that is non-complex. '


In [ ]:
import json

tfm = nussl_tfm.Compose([
    nussl_tfm.SumSources([['bass', 'drums', 'other']]),
])
test_dataset = nussl.datasets.MUSDB18(subsets=['test'], transform=tfm)

# Just do 5 items for speed. Change to 50 for actual experiment.
for i in range(5):
    item = test_dataset[i]
    separator.audio_signal = item['mix']
    estimates = separator()

    source_keys = list(item['sources'].keys())
    estimates = {
        'vocals': estimates[0],
        'bass+drums+other': item['mix'] - estimates[0]
    }

    sources = [item['sources'][k] for k in source_keys]
    estimates = [estimates[k] for k in source_keys]

    evaluator = nussl.evaluation.BSSEvalScale(
        sources, estimates, source_labels=source_keys
    )
    scores = evaluator.evaluate()
    output_folder = Path(output_folder).absolute()
    output_folder.mkdir(exist_ok=True)
    output_file = output_folder / sources[0].file_name.replace('wav', 'json')
    with open(output_file, 'w') as f:
        json.dump(scores, f, indent=4)

SeparationException: This model is not a deep audio estimation model! Did not find 'audio' key in output dictionary.

In [ ]:
import glob
import numpy as np

json_files = glob.glob(f"*.json")
df = nussl.evaluation.aggregate_score_files(
    json_files, aggregator=np.nanmedian)
nussl.evaluation.associate_metrics(separator.model, df, test_dataset)
report_card = nussl.evaluation.report_card(
    df, report_each_source=True)
print(report_card)

In [ ]:
separator.model.save('checkpoints/best.model.pth')

In [ ]:
model_checkpoint = torch.load('checkpoints/best.model.pth')

In [ ]:
model_checkpoint['metadata'].keys()

In [ ]:
model_checkpoint['metadata']['evaluation']

NameError: name 'model_checkpoint' is not defined